### GPU info

In [ ]:
!nvidia-smi

### 切換路徑

In [ ]:
cd MFGM

## 模型架構

### Lib

In [ ]:

# Util #
import pickle
import numpy as np
import os
import scipy.sparse as sp
import torch
from scipy.sparse import linalg
from torch.autograd import Variable

from sklearn.preprocessing import MinMaxScaler

# Layer #
from __future__ import division
import torch
import torch.nn as nn
from torch.nn import init
import numbers
import torch.nn.functional as F

# Model #
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys

# Trainer #
import torch.optim as optim
import math

# Main #
import torch
import numpy as np
import argparse
import time
import matplotlib.pyplot as plt


from torch.utils.tensorboard import SummaryWriter

### Util

In [ ]:

class DataLoaderM(object):
    def __init__(self, xs, ys, xs_1, xs_2, batch_size, pad_with_last_sample=True):
        """
        :param xs:
        :param ys:
        :param batch_size:
        :param pad_with_last_sample: pad with the last sample to make number of samples divisible to batch_size.
        """
        self.batch_size = batch_size
        self.current_ind = 0

        # 將資料長度補齊至batch_size可整除之數量
        # 補齊方法: 取原資料最後一個並複製多個來補齊
        if pad_with_last_sample:
            # 計算需補齊數量
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)

            # 將複製後的ele進行concatenate以補齊成可整除batch_size之長度
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)

            xs_1 = np.concatenate([xs_1, x_padding], axis=0)
            xs_2 = np.concatenate([xs_2, x_padding], axis=0)

        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
        self.ys = ys

        self.xs_1 = xs_1
        self.xs_2 = xs_2

    def shuffle(self):
        permutation = np.random.permutation(self.size)
        xs, ys = self.xs[permutation], self.ys[permutation]
        self.xs_1 = self.xs_1[permutation]
        self.xs_2 = self.xs_2[permutation]

        self.xs = xs
        self.ys = ys

    def get_iterator(self):
        self.current_ind = 0
        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = self.xs[start_ind: end_ind, ...]
                y_i = self.ys[start_ind: end_ind, ...]

                x_i_1 = self.xs_1[start_ind: end_ind, ...]
                x_i_2 = self.xs_2[start_ind: end_ind, ...]

                # 節省記憶體:
                # yield 設計來的目的，就是為了單次輸出內容
                # 我們可以把 yield 暫時看成 return，但是這個 return 的功能只有單次
                # 而且，一旦我們的程式執行到 yield 後，程式就會把值丟出，並暫時停止
                yield (x_i, y_i, x_i_1, x_i_2)
                self.current_ind += 1

        return _wrapper()

class StandardScaler():
    """
    Standard the input
    """
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
    def transform(self, data):
        return (data - self.mean) / self.std
    def inverse_transform(self, data):
        return (data * self.std) + self.mean
'''

class StandardScaler():
    """
    Standard the input
    """
    def __init__(self, max, min):
        self.max = max
        self.min = min
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    def inverse_transform(self, data):
        return (data * (self.max - self.min) ) + self.min
'''

def asym_adj(adj):
    """Asymmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)).flatten()
    d_inv = np.power(rowsum, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat= sp.diags(d_inv)
    return d_mat.dot(adj).astype(np.float32).todense()


def load_pickle(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data

def load_adj(pkl_filename, adjtype):
    sensor_ids, sensor_id_to_ind, adj_mx = load_pickle(pkl_filename)

    print('# 全部L.A.的sensor ID(sensor_ids):\n',sensor_ids)
    print('# 將sensor ID對應index(sensor_id_to_ind):\n',sensor_id_to_ind)

    if adjtype == "scalap":
        adj = [calculate_scaled_laplacian(adj_mx)]
    elif adjtype == "normlap":
        adj = [calculate_normalized_laplacian(adj_mx).astype(np.float32).todense()]
    elif adjtype == "symnadj":
        adj = [sym_adj(adj_mx)]
    elif adjtype == "transition":
        adj = [asym_adj(adj_mx)]
    elif adjtype == "doubletransition":
        adj = [asym_adj(adj_mx), asym_adj(np.transpose(adj_mx))]   # asym_adj(adj_mx): forward transition matrix / asym_adj(np.transpose(adj_mx)): backward transition matrix
    elif adjtype == "identity":
        adj = [np.diag(np.ones(adj_mx.shape[0])).astype(np.float32)]
    else:
        error = 0
        assert error, "adj type not defined"

    print('# Double transition Transition matrix of Eq 4:\n',adj)
    return sensor_ids, sensor_id_to_ind, adj

def load_dataset(dataset_dir, batch_size, valid_batch_size= None, test_batch_size=None):
    data = {}
    for category in ['train', 'val', 'test']:
        cat_data = np.load(os.path.join(dataset_dir, category + '.npz'))
        data['x_' + category] = cat_data['x']
        data['y_' + category] = cat_data['y']

        if args.log_print:
            print("# category:", category)
            print('x:',data['x_' + category].shape, data['x_' + category][0] )
            print('y:',data['y_' + category].shape, data['y_' + category][0] )

    # 使用train的mean/std來正規化valid/test #
    scaler = StandardScaler(mean=data['x_train'][..., 0].mean(), std=data['x_train'][..., 0].std())
    # 將欲訓練特徵改成正規化
    for category in ['train', 'val', 'test']:
        data['x_' + category][..., 0] = scaler.transform(data['x_' + category][..., 0])



    data['train_loader'] = DataLoaderM(data['x_train'], data['y_train'], batch_size)
    data['val_loader'] = DataLoaderM(data['x_val'], data['y_val'], valid_batch_size)
    data['test_loader'] = DataLoaderM(data['x_test'], data['y_test'], test_batch_size)
    data['scaler'] = scaler
    return data



def masked_mse(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = (preds-labels)**2
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

def masked_rmse(preds, labels, null_val=np.nan):
    return torch.sqrt(masked_mse(preds=preds, labels=labels, null_val=null_val))


def masked_mae(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds-labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)
def masked_mape(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds-labels)/labels
    #loss = 2.0 * torch.mean(torch.abs(preds - labels) / (torch.abs(preds) + torch.abs(labels)))
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)
def masked_smape(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    #loss = torch.abs(preds-labels)/labels
    loss = 2.0 * (torch.abs(preds - labels) / (torch.abs(preds) + torch.abs(labels)))
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

def metric(pred, real):
    mae = masked_mae(pred,real,0.0).item()
    mape = masked_mape(pred,real,0.0).item()
    rmse = masked_rmse(pred,real,0.0).item()
    smape = masked_smape(pred,real,0.0).item()
    return mae,mape,rmse,smape

### Layer

In [ ]:

class nconv(nn.Module):
    def __init__(self):
        super(nconv,self).__init__()

    def forward(self,x, A):
        x = torch.einsum('ncwl,vw->ncvl',(x,A))  # adj的row為鄰接=>橫向聚合
        return x.contiguous()


class linear(nn.Module):
    def __init__(self,c_in,c_out,bias=True):
        super(linear,self).__init__()
        self.mlp = torch.nn.Conv2d(c_in, c_out, kernel_size=(1, 1), padding=(0,0), stride=(1,1), bias=bias)

    def forward(self,x):
        return self.mlp(x)


class LayerNorm(nn.Module):
    __constants__ = ['normalized_shape', 'weight', 'bias', 'eps', 'elementwise_affine']
    def __init__(self, normalized_shape, eps=1e-5, elementwise_affine=True):
        super(LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        self.normalized_shape = tuple(normalized_shape)
        self.eps = eps
        self.elementwise_affine = elementwise_affine
        if self.elementwise_affine:
            self.weight = nn.Parameter(torch.Tensor(*normalized_shape))
            self.bias = nn.Parameter(torch.Tensor(*normalized_shape))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)
        self.reset_parameters()


    def reset_parameters(self):
        if self.elementwise_affine:
            init.ones_(self.weight)
            init.zeros_(self.bias)

    def forward(self, input, idx):
        if self.elementwise_affine:
            return F.layer_norm(input, tuple(input.shape[1:]), self.weight[:,idx,:], self.bias[:,idx,:], self.eps)
        else:
            return F.layer_norm(input, tuple(input.shape[1:]), self.weight, self.bias, self.eps)

    def extra_repr(self):
        return '{normalized_shape}, eps={eps}, ' \
            'elementwise_affine={elementwise_affine}'.format(**self.__dict__)

### Multivariate View

In [ ]:


# this efficient implementation comes from https://github.com/xptree/DeepInf/
class F_BatchMultiHeadGraphAttention(nn.Module):
    def __init__(self, n_heads, num_nodes, dropout, bias=True):
        super(F_BatchMultiHeadGraphAttention, self).__init__()

        print('F_BatchMultiHeadGraphAttention', n_heads, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        #self.w = nn.Parameter(torch.Tensor(self.n_head, num_nodes, num_nodes))
        self.a_src = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))

        self.mlp_convs = (nn.Conv2d(n_heads, n_heads, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_nodes))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)

        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

    def forward(self, h):
        bs, ch, n, dim = h.size()
        h_prime = (h)
        attn_src = torch.matmul(torch.tanh(h), self.a_src)
        attn_dst = torch.matmul(torch.tanh(h), self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )
        attn = self.leaky_relu(attn)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.matmul(attn, h_prime)
        return output, attn


class Fusion(nn.Module):
    def __init__(self, kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout):
        super(Fusion, self).__init__()

        self.gat_layer = F_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )
        self.gat_layer2 = F_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )

        self.mlp1 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))
        self.mlp2 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))
        self.mlp3 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))
        self.mlp4 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))
        self.mlp5 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))
        self.mlp6 = (nn.Conv2d(32,8,(1, 1),dilation=(1,1)))


        self.mlp7 = (nn.Conv2d(16,32,(1, 1),dilation=(1,1)))
    def forward(self,x_input, diff1, diff2, x1, x2):

        bs,c,n,t = x_input.shape
        x_input_cpy = x_input

        x_input1 = self.mlp1(x_input)
        diff1 = self.mlp2(diff1)
        diff2 = self.mlp3(diff2)

        x_input2 = self.mlp4(x_input)
        x1 = self.mlp5(x1)
        x2 = self.mlp6(x2)

        bs,c,n,t = x1.shape
        x_input_new = []
        x1_all = []
        x2_all = []
        for i in range(t):
          x_t = (torch.cat([x_input1[:,:,:,[i]], diff1[:,:,:,[i]], diff2[:,:,:,[i]]], dim=3)).permute(0,1,3,2)
          x_all, attn = self.gat_layer( x_t )


          x_t = (torch.cat([x_input2[:,:,:,[i]], x1[:,:,:,[i]], x2[:,:,:,[i]]], dim=3)).permute(0,1,3,2)
          x_all2, attn = self.gat_layer2( x_t )

          x1_all.append(x_all[:,:,[0]].permute(0,1,3,2))
          x2_all.append(x_all2[:,:,[0]].permute(0,1,3,2))

        x1 = torch.cat(x1_all,dim=3)
        x2 = torch.cat(x2_all,dim=3)

        x_out = self.mlp7( torch.cat([x1,x2],dim=1) )+x_input_cpy


        return x_out

### Temporal View

In [ ]:
import numpy as np

def create_matrix(n, k):
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(k):
            if i-j >= 0:
                mat[i][i-j] = 1
    return mat


# this efficient implementation comes from https://github.com/xptree/DeepInf/
class BatchMultiHeadGraphAttention(nn.Module):
    def __init__(self, n_heads, num_nodes, dropout, bias=True):
        super(BatchMultiHeadGraphAttention, self).__init__()

        print('BatchMultiHeadGraphAttention', n_heads, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        #self.w = nn.Parameter(torch.Tensor(self.n_head, num_nodes, num_nodes))
        self.a_src = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_nodes))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)

        #nn.init.xavier_uniform_(self.w, gain=1.414)
        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

    def forward(self, h):

        bs, ch, n, dim = h.size()
        #h_prime = torch.matmul(h, self.w)
        h_prime = h
        attn_src = torch.matmul(h, self.a_src)
        attn_dst = torch.matmul(h, self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )

        attn = self.leaky_relu(attn)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.matmul(attn, h_prime)

        return output, attn

# MutiChannel_GAT(kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout)
class MutiChannel_GAT(nn.Module):
    def __init__(self, kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout):
        super(MutiChannel_GAT, self).__init__()

        print('MutiChannel_GAT', n_heads, num_nodes, dropout)

        self.gat_layer1 = BatchMultiHeadGraphAttention(
            n_heads*2, num_nodes, dropout
        )

        self.gat_layer4 = BatchMultiHeadGraphAttention(
            n_heads*2, num_nodes, dropout
        )

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

        self.mlp1 = (nn.Conv2d(32,16,(2, 1),dilation=(1,1)))
        self.mlp4 = (nn.Conv2d(32,16,(5, 1),dilation=(1,1)))

    def forward(self,x_input):

        x_input_cpy = x_input

        x_input1 = self.mlp1(x_input)
        x_input4 = self.mlp4(x_input)

        #-------------GAT-------------#
        x_input1, attn = self.gat_layer1(x_input1)
        x_input4, attn = self.gat_layer4(x_input4)
        #-------------GAT-------------#

        x_input = torch.cat([x_input1[:,:,-x_input4.size(2):],x_input4[:,:,-x_input4.size(2):]], dim=1)

        x_input = ((x_input_cpy)[:,:,-x_input.size(2):] + (x_input)).permute(0,1,3,2)

        return x_input


### Spatial View

In [ ]:
import numpy as np

def create_matrix(n, k):
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(k):
            if i-j >= 0:
                mat[i][i-j] = 1
    return mat


def pearson_corr2(tensor): # all: (64,1,n,dim)
  # Input tensor shape: (batch_size, num_nodes, time_steps)
  batch_size, num_nodes, _ = tensor.shape
  tensor = tensor - tensor.mean(dim=2, keepdim=True)
  std = tensor.std(dim=2, keepdim=True)
  tensor = tensor / (std + 1e-8)
  correlation_matrix = torch.matmul(tensor, tensor.transpose(1, 2))
  correlation_matrix = correlation_matrix / (tensor.shape[2] - 1)
  return correlation_matrix


def topK(attn, top_num ):

  # Get the top K values and their indices for each row
  top_k_values, top_k_indices = attn.topk(top_num, dim=3)

  # Create a mask with the same shape as the input tensor, filled with zeros
  mask = torch.zeros_like(attn)

  # Set the top K values in the mask to 1
  mask.scatter_(3, top_k_indices, 1)

  # Multiply the input tensor with the mask to get the result
  attn = attn * mask

  return  attn

# this efficient implementation comes from https://github.com/xptree/DeepInf/
class S_BatchMultiHeadGraphAttention(nn.Module):
    def __init__(self, n_heads, num_nodes, dropout, bias=True):
        super(S_BatchMultiHeadGraphAttention, self).__init__()

        print('S_BatchMultiHeadGraphAttention', n_heads, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        self.w = nn.Parameter(torch.Tensor(self.n_head*2, 1, 40))
        self.w2 = nn.Parameter(torch.Tensor(self.n_head*2, 40, 1))

        self.a_src = nn.Parameter(torch.Tensor(self.n_head*2, 40, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head*2, 40, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_nodes))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)

        nn.init.xavier_uniform_(self.w, gain=1.414)
        nn.init.xavier_uniform_(self.w2, gain=1.414)
        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

    def forward(self, h, a):


        bs, ch, n, dim = h.size()
        attn_src = torch.matmul(self.leaky_relu(torch.matmul(h, self.w)), self.a_src)
        attn_dst = torch.matmul(self.leaky_relu(torch.matmul(h, self.w)), self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )


        attn_2 = self.leaky_relu(attn)
        attn_2 = self.softmax(attn_2)
        attn_2 = self.dropout(attn_2)

        if len(a)>0:

          attn_2 = (attn_2+a)/2
        else:
          attn_2 = attn_2

        output_2 = torch.matmul(attn_2, h)

        return output_2, attn

# MutiChannel_GAT(kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout)
class S_MutiChannel_GAT(nn.Module):
    def __init__(self, kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout):
        super(S_MutiChannel_GAT, self).__init__()

        print('S_MutiChannel_GAT', n_heads, num_nodes, dropout)

        self.gat_layer = S_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )
        self.gat_layer2 = S_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )


        self.mlp1 =  nn.Conv2d(in_channels=32,
                                    out_channels=16,
                                    kernel_size=(1, 1))

        self.mlp2 =  nn.Conv2d(in_channels=16*3,
                                    out_channels=16,
                                    kernel_size=(1, 1))

    def forward(self,x_input, a_f, a_b):
        x_input_cpy = x_input

        x = x_input

        xf= x_input

        bs,c,n,t = xf.shape
        x1_all = []
        x2_all = []
        x3_all = []
        x4_all = []
        for i in range(t):
          x_in = xf[...,[i]]

          x1, attn = self.gat_layer(x_in,a_f)

          x2, attn = self.gat_layer2((x1),[])
          x1_all.append(x1)
          x2_all.append(x2)


        x1 = torch.cat(x1_all,dim=3)
        x2 = torch.cat(x2_all,dim=3)

        x_out = self.mlp2(torch.cat([xf,x1,x2],dim=1))

        return x_out

### Model

In [ ]:
class MFGM(nn.Module):
    def __init__(self, model_type, gcn_true, buildA_true, gcn_depth, num_nodes, device, predefined_A=None,kernel_set=None, static_feat=None, dropout=0.3, subgraph_size=20, node_dim=40, dilation_exponential=1, conv_channels=32, residual_channels=32, skip_channels=64, end_channels=128, seq_length=12, in_dim=2, out_dim=12, layers=3, propalpha=0.05, tanhalpha=3, layer_norm_affline=True):
        super(MFGM, self).__init__()

        self.model_type = model_type

        self.gcn_true = gcn_true
        self.buildA_true = buildA_true
        self.num_nodes = num_nodes
        self.dropout = dropout
        self.predefined_A = predefined_A
        self.layers = layers
        self.seq_length = seq_length

        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        #----------------------#
        self.filter_convs1 = nn.ModuleList()
        self.gate_convs1 = nn.ModuleList()
        self.filter_convs2 = nn.ModuleList()
        self.gate_convs2 = nn.ModuleList()
        #----------------------#


        self.residual_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        self.gconv1 = nn.ModuleList()
        self.gconv2 = nn.ModuleList()
        self.norm = nn.ModuleList()

        #----------------------#
        self.gconv1_1 = nn.ModuleList()
        self.gconv1_2 = nn.ModuleList()

        self.gconv2_1 = nn.ModuleList()
        self.gconv2_2 = nn.ModuleList()

        self.norm1 = nn.ModuleList()
        self.norm2 = nn.ModuleList()

        self.fusion = nn.ModuleList()
        #----------------------#

        self.start_conv = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))
        self.start_conv1 = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))
        self.start_conv2 = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))

        self.start_conv3 = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))
        self.start_conv4 = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))

        dilation_factor = 1
        n_heads = 8
        kern = 6
        self.fusion = (Fusion(kern, dilation_factor, n_heads, 21, [24,16,8], [24,32], dropout))

        # Paepr eq 11: R=1+(c-1)(q^m -1)/(q -1).
        kernel_size = 7
        if dilation_exponential>1:
            self.receptive_field = int(1+(kernel_size-1)*(dilation_exponential**layers-1)/(dilation_exponential-1))
        else:
            self.receptive_field = layers*(kernel_size-1) + 1

        print("# Model Type", self.model_type)
        print("# receptive_field", self.receptive_field)
        i=0
        if dilation_exponential>1:
            rf_size_i = int(1 + i*(kernel_size-1)*(dilation_exponential**layers-1)/(dilation_exponential-1))
        else:
            rf_size_i = i*layers*(kernel_size-1)+1
        new_dilation = 1

        self.receptive_field = 13
        temporal_len = self.receptive_field
        for j in range(1,layers+1):

            if dilation_exponential > 1:
                rf_size_j = int(rf_size_i + (kernel_size-1)*(dilation_exponential**j-1)/(dilation_exponential-1))
            else:
                rf_size_j = rf_size_i+j*(kernel_size-1)

            kern = 5
            dilation_factor = 1
            n_heads = 8
            #num_nodes = temporal_len
            print('temporal_len', temporal_len)
            self.filter_convs.append(MutiChannel_GAT(kern, dilation_factor, n_heads, num_nodes, [24,16,8], [32,32], dropout))
            self.gate_convs.append(MutiChannel_GAT(kern, dilation_factor, n_heads, num_nodes, [24,16,8], [32,32], dropout))


            temporal_len = temporal_len-(kern-1)

            '''
            # skip_convs #
            (0): Conv2d(32, 64, kernel_size=(1, 13), stride=(1, 1))
            (1): Conv2d(32, 64, kernel_size=(1, 7), stride=(1, 1))
            (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
            '''
            if self.seq_length>self.receptive_field:
                self.skip_convs.append(nn.Conv2d(in_channels=conv_channels,
                                                out_channels=skip_channels,
                                                kernel_size=(1, temporal_len)))
            else:
                self.skip_convs.append(nn.Conv2d(in_channels=conv_channels,
                                                out_channels=skip_channels,
                                                kernel_size=(1, temporal_len)))
            dilation_factor = 1
            n_heads = 8

            self.gconv1.append(S_MutiChannel_GAT(kern, dilation_factor, n_heads, temporal_len, [24,16,8], [16,24,32], dropout))
            self.gconv2.append(S_MutiChannel_GAT(kern, dilation_factor, n_heads, temporal_len, [24,16,8], [16,24,32], dropout))




            #####   Normalization   ##### START
            if self.seq_length>self.receptive_field:
                print('1', self.seq_length - rf_size_j + 1)
                self.norm.append(LayerNorm((residual_channels, num_nodes, temporal_len),elementwise_affine=layer_norm_affline))

            else:
                print('2', self.receptive_field - rf_size_j + 1)
                self.norm.append(LayerNorm((residual_channels, num_nodes, temporal_len),elementwise_affine=layer_norm_affline))

            #####   Normalization   ##### END




            new_dilation *= dilation_exponential



        self.end_conv_1 = nn.Conv2d(in_channels=skip_channels,
                                             out_channels=end_channels,
                                             kernel_size=(1,1),
                                             bias=True)
        self.end_conv_2 = nn.Conv2d(in_channels=end_channels,
                                             out_channels=out_dim,
                                             kernel_size=(1,1),
                                             bias=True)

        #####   SKIP layer   ##### START
        '''
        (skip0): Conv2d(2, 64, kernel_size=(1, 19), stride=(1, 1))
        (skipE): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        '''
        if self.seq_length > self.receptive_field:
            self.skip0 = nn.Conv2d(in_channels=in_dim, out_channels=skip_channels, kernel_size=(1, self.seq_length), bias=True)
            self.skipE = nn.Conv2d(in_channels=residual_channels, out_channels=skip_channels, kernel_size=(1, self.seq_length-self.receptive_field+1), bias=True)

        else:
            self.skip0 = nn.Conv2d(in_channels=in_dim, out_channels=skip_channels, kernel_size=(1, self.receptive_field), bias=True)
            self.skipE = nn.Conv2d(in_channels=residual_channels, out_channels=skip_channels, kernel_size=(1, 1), bias=True)
        #####   SKIP layer   ##### END

        self.idx = torch.arange(self.num_nodes).to(device)


    def forward(self, input, input_1, input_2, idx=None):
        seq_len = input.size(3)
        assert seq_len==self.seq_length, 'input sequence length not equal to preset sequence length'


        # Step0: 檢查receptive_field, 不足則padding0
        if self.seq_length<self.receptive_field:
            input = nn.functional.pad(input,(self.receptive_field-self.seq_length,0,0,0))
            input_1 = nn.functional.pad(input_1,(self.receptive_field-self.seq_length,0,0,0))
            input_2 = nn.functional.pad(input_2,(self.receptive_field-self.seq_length,0,0,0))

        # Step1: turn([64, 2, 207, 19]) to ([64, 32, 207, 19])
        x = self.start_conv(input)

        diff_1 = torch.cat([(input[:,[0]]-input_1[:,[0]]),input[:,[1]]], dim=1)
        diff_2 = torch.cat([(input[:,[0]]-input_2[:,[0]]),input[:,[1]]], dim=1)

        diff_1 = self.start_conv1(diff_1)
        diff_2 = self.start_conv2(diff_2)

        x1 = self.start_conv3(input_1)
        x2 = self.start_conv4(input_2)

        x = self.fusion(x,diff_1,diff_2,x1,x2)

        # Step1-1: original input skip =>(skip0)
        # (skip0): Conv2d(2, 64, kernel_size=(1, 19), stride=(1, 1))
        # ([64, 32, 207, 19]) -> ([64, 64, 207, 1])
        skip = self.skip0(F.dropout(input, self.dropout, training=self.training))


        # Layers : 3層 : 19->13->7->1 (取決於TCN取的維度)
        for i in range(self.layers):

            # Step2: Temporal Model --START #
            # 為上一層輸出, ex:  [64, 32, 207, 19] -> [64, 32, 207, 13] -> [64, 32, 207, 7]-> [64, 32, 207, 1]
            residual = x


            x = x.permute(0,1,3,2)
            # Tanh
            filter = self.filter_convs[i](x)
            filter = torch.tanh(filter)

            # Sigmoid
            gate = self.gate_convs[i](x)
            gate = torch.sigmoid(gate)

            # Fusion
            x = filter * gate

            #-----------------#


            x = F.dropout(x, self.dropout, training=self.training)

            s = x
            #s = self.fusion[i](x,x1,x2)
            s = self.skip_convs[i](s)

            skip = s + skip

            # Step3: Skip after TCN --END #

            x = torch.cat([self.gconv1[i](x[:,:16], self.predefined_A[0], self.predefined_A[1]), self.gconv2[i](x[:,16:32], self.predefined_A[0], self.predefined_A[1])], dim=1 )

            x = x + residual[:, :, :, -x.size(3):]


            if idx is None:
                x = self.norm[i](x,self.idx)

            else:
                x = self.norm[i](x,idx)

            # Step4: GCN --END #

        #(skipE): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        skip = self.skipE(x) + skip

        x = F.relu(skip)
        x = F.relu(self.end_conv_1(x))
        x = self.end_conv_2(x)
        return x

### Trainer

In [ ]:
class Trainer():
    def __init__(self, model, lrate, wdecay, clip, step_size, seq_out_len, scaler, device, cl=True):
        self.scaler = scaler
        self.model = model
        self.model.to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lrate, weight_decay=wdecay)
        self.loss = masked_mae
        self.clip = clip

        self.step = step_size
        self.iter = 1
        self.task_level = 1
        self.seq_out_len = seq_out_len
        self.cl = cl

    def train(self, input, real_val, input_1, input_2, idx=None):
        self.model.train()
        self.optimizer.zero_grad()
        output = self.model(input, input_1, input_2, idx=idx)
        output = output.transpose(1,3)
        real = torch.unsqueeze(real_val,dim=1)

        predict = self.scaler.inverse_transform(output)

        if self.iter%self.step==0 and self.task_level<=self.seq_out_len:
            self.task_level +=1
            print("### cl learning\n iter",self.iter,"\niter%step",self.iter%self.step,"\ntask_level",self.task_level)
            print("# predict len:", len(predict[:, :, :, :self.task_level]))

        if self.cl:
            loss = masked_mae(predict[:, :, :, :self.task_level], real[:, :, :, :self.task_level], 0.0)
        else:
            loss = masked_mae(predict, real, 0.0)

        loss.backward()

        if self.clip is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)

        self.optimizer.step()
        mae = masked_mae(predict,real,0.0).item()
        mape = masked_mape(predict,real,0.0).item()
        rmse = masked_rmse(predict,real,0.0).item()
        smape = masked_smape(predict,real,0.0).item()
        self.iter += 1
        return mae,mape,rmse,smape

    def eval(self, input,  real_val, input_1, input_2):
        self.model.eval()

        output = self.model(input, input_1, input_2)
        output = output.transpose(1,3)
        real = torch.unsqueeze(real_val,dim=1)
        #predict = self.scaler.inverse_transform(output)
        #predict = self.scaler.inverse_transform(output)

        '''
        predict = output

        for i in range(args.num_nodes):
          predict[:,0,i,:] = self.scaler[i].inverse_transform(predict[:,0,i,:])
        '''
        predict = self.scaler.inverse_transform(output)

        loss = self.loss(predict, real, 0.0)
        mape = masked_mape(predict,real,0.0).item()
        rmse = masked_rmse(predict,real,0.0).item()
        smape = masked_smape(predict,real,0.0).item()
        return loss.item(),mape,rmse,smape


### Parameter

In [ ]:


def str_to_bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in {'false', 'f', '0', 'no', 'n'}:
        return False
    elif value.lower() in {'true', 't', '1', 'yes', 'y'}:
        return True
    raise ValueError(f'{value} is not a valid boolean value')


parser = argparse.ArgumentParser()
parser.add_argument('--device',type=str,default='cuda',help='')
parser.add_argument('--adjtype',type=str,default='doubletransition',help='adj type')


parser.add_argument('--gcn_true', type=str_to_bool, default=True, help='whether to add graph convolution layer')
parser.add_argument('--buildA_true', type=str_to_bool, default=True,help='whether to construct adaptive adjacency matrix')

parser.add_argument('--cl', type=str_to_bool, default=True,help='whether to do curriculum learning')

parser.add_argument('--gcn_depth',type=int,default=2,help='graph convolution depth')

parser.add_argument('--dropout',type=float,default=0.3,help='dropout rate')
parser.add_argument('--subgraph_size',type=int,default=20,help='k')
parser.add_argument('--node_dim',type=int,default=40,help='dim of nodes')
parser.add_argument('--dilation_exponential',type=int,default=1,help='dilation exponential')

parser.add_argument('--conv_channels',type=int,default=32,help='convolution channels')
parser.add_argument('--residual_channels',type=int,default=32,help='residual channels')


parser.add_argument('--in_dim',type=int,default=2,help='inputs dimension')
parser.add_argument('--seq_in_len',type=int,default=12,help='input sequence length')
parser.add_argument('--seq_out_len',type=int,default=12,help='output sequence length')


parser.add_argument('--batch_size',type=int,default=64,help='batch size')

parser.add_argument('--clip',type=int,default=5,help='clip')


parser.add_argument('--propalpha',type=float,default=0.05,help='prop alpha')
parser.add_argument('--tanhalpha',type=float,default=3,help='adj alpha')


parser.add_argument('--model_type',type=str,default='MFGM',help='model type')
parser.add_argument('--skip_channels',type=int,default=64,help='skip channels')
parser.add_argument('--end_channels',type=int,default=128,help='end channels')
parser.add_argument('--layers',type=int,default=3,help='number of layers')
parser.add_argument('--kernel_set',default=[2,5], type=int, nargs='+')



parser.add_argument('--print_every',type=int,default=50,help='')
parser.add_argument('--seed',type=int,default=101,help='random seed')
parser.add_argument('--save',type=str,default='./save/',help='save path')

parser.add_argument('--log_print', type=str_to_bool, default=False ,help='whether to load static feature')

parser.add_argument('--learning_rate',type=float,default=0.0005,help='learning rate')
parser.add_argument('--weight_decay',type=float,default=0.0001,help='weight decay rate')

parser.add_argument('--step_size1',type=int,default=800,help='step_size')
parser.add_argument('--step_size2',type=int,default=100,help='step_size')

target = 'hsin-2022-21nodes'
parser.add_argument('--data',type=str,default='../Data/'+target ,help='data path')
parser.add_argument('--adj_data',type=str,default='../Data/'+target+'/adj_mat_2022_hsin_21_locs.pkl',help='adj data path')
parser.add_argument('--num_nodes',type=int,default=21,help='number of nodes/variables')

parser.add_argument('--expid',type=int,default=202304201422,help='experiment id')
parser.add_argument('--runs',type=int,default=10,help='number of runs')
parser.add_argument('--epochs',type=int,default=200,help='')

#args = parser.parse_args()
args=parser.parse_args(args=[])
torch.set_num_threads(3)

#args = parser.parse_args()
args=parser.parse_args(args=[])
print('# args', args)

device = torch.device(args.device)

writer = SummaryWriter()

### Loading Data

In [ ]:

batch_size = args.batch_size
valid_batch_size = args.batch_size
test_batch_size = args.batch_size
data = {}

type_list = ['vehicular', 'stationary', 'pedestrian']

for i in range(len(type_list)):

  for category in ['train', 'val', 'test']:

      # Loading npz
      cat_data = np.load(os.path.join(args.data, category+"_"+type_list[i] + '.npz'))
      print("loading:", category+"_"+type_list[i] + '.npz')

      if i == 0:
        key = ""
      else:
        key = "_"+str(i)
      data['x_' + category+key] = cat_data['x']     # (?, 12, 207, 2)
      data['y_' + category+key] = cat_data['y']     # (?, 12, 207, 2)

print(data.keys())


# 使用train的mean/std來正規化valid/test #
scaler = StandardScaler(mean=data['x_train'][..., 0].mean(), std=data['x_train'][..., 0].std())

# 將欲訓練特徵改成正規化
for i in range(len(type_list)):
  if i == 0:
    key = ""
  else:
    key = "_"+str(i)

  for category in ['train', 'val', 'test']:
    data['x_' + category+key][..., 0] = scaler.transform(data['x_' + category+key][..., 0])


data['train_loader'] = DataLoaderM(data['x_train'], data['y_train'], data['x_train_1'], data['x_train_2'], batch_size)
data['val_loader'] = DataLoaderM(data['x_val'], data['y_val'], data['x_val_1'], data['x_val_2'], valid_batch_size)
data['test_loader'] = DataLoaderM(data['x_test'], data['y_test'], data['x_test_1'], data['x_test_2'], test_batch_size)
data['scaler'] = scaler

'''
adj_mx: 根據distances_la_2012.csv, 找出每個sensor與其他sensor距離並建立距離矩陣, 再進行正規化
'''
sensor_ids, sensor_id_to_ind, adj_mx = load_adj(args.adj_data,args.adjtype)   # adjtype: default='doubletransition'

adj_mx = [torch.tensor(i).to(device) for i in adj_mx]

dataloader = data.copy()


### Main

In [ ]:


def main(runid):


    # if args.load_static_feature:
    #     static_feat = load_node_feature('data/sensor_graph/location.csv')
    # else:
    #     static_feat = None

    model = MFGM(args.model_type, args.gcn_true, args.buildA_true, args.gcn_depth, args.num_nodes,
                   device, predefined_A=adj_mx, kernel_set=args.kernel_set, dropout=args.dropout, subgraph_size=args.subgraph_size, node_dim=args.node_dim, dilation_exponential=args.dilation_exponential, conv_channels=args.conv_channels, residual_channels=args.residual_channels,
                  skip_channels=args.skip_channels, end_channels= args.end_channels,
                  seq_length=args.seq_in_len, in_dim=args.in_dim, out_dim=args.seq_out_len,
                  layers=args.layers, propalpha=args.propalpha, tanhalpha=args.tanhalpha, layer_norm_affline=True)

    print(model)
    print(args)

    print('The recpetive field size is', model.receptive_field)
    nParams = sum([p.nelement() for p in model.parameters()])       # model參數量!
    print('Number of model parameters is', nParams)

    engine = Trainer(model, args.learning_rate, args.weight_decay, args.clip, args.step_size1, args.seq_out_len, data['scaler'], device, args.cl)

    print("start training...",flush=True)
    his_loss =[]
    val_time = []
    train_time = []
    minl = 1e5
    start_epoch=0
    SAVE_PATH = ""
    train_loss_epoch = []  # 紀錄train在epoch收斂
    valid_loss_epoch = []  # 紀錄valid在epoch收斂
    """
    ########------------------- 讀取檔案要處理的code -----------------------#############
    ### loading model ###
    SAVE_PATH = args.save + "exp"+str(args.expid)+"_0.pth"

    checkpoint = torch.load(SAVE_PATH)
    engine.model.load_state_dict(checkpoint['model_state_dict'])
    engine.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    engine.task_level = checkpoint['task_level']
    start_epoch = checkpoint['epoch']
    #minl = checkpoint['loss']
    train_loss_epoch = checkpoint['train_loss']
    valid_loss_epoch = checkpoint['valid_loss']
    ### 測試讀取出的model ###
    valid_loss = []
    valid_mape = []
    valid_rmse = []
    for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)
        testy = torch.Tensor(y).to(device)
        testy = testy.transpose(1, 3)
        metrics = engine.eval(testx, testy[:,0,:,:])
        valid_loss.append(metrics[0])
        valid_mape.append(metrics[1])
        valid_rmse.append(metrics[2])

    mvalid_loss = np.mean(valid_loss)
    mvalid_mape = np.mean(valid_mape)
    mvalid_rmse = np.mean(valid_rmse)
    print("### 2-The valid loss on loding model is", str(round(mvalid_mape,4)))
    minl= mvalid_mape
    print("### minl:",minl, "checkpoint['loss']:",checkpoint['loss'])
    ### 測試讀取出的model ###
    ########------------------- 讀取檔案要處理的code -----------------------#############
    """
    '''
    #####
    SAVE_PATH = args.save + "exp202111182244_0.pth"
    checkpoint = torch.load(SAVE_PATH)
    engine.model.load_state_dict(checkpoint['model_state_dict'])
    engine.task_level = checkpoint['task_level']
    start_epoch = checkpoint['epoch']
    train_loss_epoch = checkpoint['train_loss']
    valid_loss_epoch = checkpoint['valid_loss']
    #####
    '''
    for i in range(start_epoch,start_epoch+args.epochs+1):

        train_loss = []
        train_mape = []
        train_rmse = []
        train_smape = []
        t1 = time.time()
        dataloader['train_loader'].shuffle()  # 為了檢視資料先拿掉
        for iter, (x, y, x_1, x_2) in enumerate(dataloader['train_loader'].get_iterator()):
            trainx = torch.Tensor(x).to(device)
            trainx= trainx.transpose(1, 3)
            trainy = torch.Tensor(y).to(device)
            trainy = trainy.transpose(1, 3)

            trainx_1 = torch.Tensor(x_1).to(device)
            trainx_1= trainx_1.transpose(1, 3)

            trainx_2 = torch.Tensor(x_2).to(device)
            trainx_2= trainx_2.transpose(1, 3)

            metrics = engine.train(trainx, trainy[:,0,:,:], trainx_1, trainx_2)

            train_loss.append(metrics[0])
            train_mape.append(metrics[1])
            train_rmse.append(metrics[2])
            train_smape.append(metrics[3])

            if iter % args.print_every == 0 :
                log = 'Iter: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}'
                print(log.format(iter, train_loss[-1], train_mape[-1], train_rmse[-1]),flush=True)
        t2 = time.time()
        train_time.append(t2-t1)
        #validation
        valid_loss = []
        valid_mape = []
        valid_rmse = []
        valid_smape = []

        s1 = time.time()
        for iter, (x, y, x_1, x_2) in enumerate(dataloader['val_loader'].get_iterator()):
            testx = torch.Tensor(x).to(device)
            testx = testx.transpose(1, 3)
            testy = torch.Tensor(y).to(device)
            testy = testy.transpose(1, 3)

            testx_1 = torch.Tensor(x_1).to(device)
            testx_1 = testx_1.transpose(1, 3)
            testx_2 = torch.Tensor(x_2).to(device)
            testx_2 = testx_2.transpose(1, 3)

            metrics = engine.eval(testx, testy[:,0,:,:], testx_1, testx_2)

            valid_loss.append(metrics[0])
            valid_mape.append(metrics[1])
            valid_rmse.append(metrics[2])
            valid_smape.append(metrics[3])
        s2 = time.time()
        log = 'Epoch: {:03d}, Inference Time: {:.4f} secs'
        print(log.format(i,(s2-s1)))
        val_time.append(s2-s1)
        mtrain_loss = np.mean(train_loss)
        mtrain_mape = np.mean(train_mape)
        mtrain_rmse = np.mean(train_rmse)
        mtrain_smape = np.mean(train_smape)

        mvalid_loss = np.mean(valid_loss)
        mvalid_mape = np.mean(valid_mape)
        mvalid_rmse = np.mean(valid_rmse)
        mvalid_smape = np.mean(valid_smape)
        #his_loss.append(mvalid_loss)
        his_loss.append(mvalid_smape)

        #writer.add_scalar("train_loss", mtrain_loss, i)
        #writer.add_scalar("valid_loss", mvalid_loss, i)

        writer.add_scalar("train_loss", mvalid_loss, i)
        writer.add_scalar("valid_loss", mvalid_loss, i)


        log = 'Epoch: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}, Valid Loss: {:.4f}, Valid MAPE: {:.4f}, Valid RMSE: {:.4f}, Training Time: {:.4f}/epoch'
        print(log.format(i, mtrain_loss, mtrain_mape, mtrain_rmse, mvalid_loss, mvalid_mape, mvalid_rmse, (t2 - t1)),flush=True)
        # 紀錄每個epoch的loss
        train_loss_epoch.append(mtrain_loss)
        valid_loss_epoch.append(mvalid_loss)

        '''
        if mvalid_loss<minl:
            torch.save(engine.model.state_dict(), args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth")
            minl = mvalid_loss
        '''
        if mvalid_loss<minl:
            target_best_model = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
            print("### Update Best Model:",target_best_model, 'Loss:', mvalid_mape, " ###")
            #torch.save(engine.model.state_dict(), args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth")
            SAVE_PATH = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
            torch.save({
              'epoch': i,
              'task_level': engine.task_level,
              'model_state_dict': engine.model.state_dict(),
              'optimizer_state_dict': engine.optimizer.state_dict(),
              'loss': mvalid_mape,
              'train_loss': train_loss_epoch,
              'valid_loss': valid_loss_epoch
            }, SAVE_PATH)
            minl = mvalid_loss

    print("Average Training Time: {:.4f} secs/epoch".format(np.mean(train_time)))
    print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))


    bestid = np.argmin(his_loss)

    writer.close()
    print("Training finished")
    print("The valid loss on best model is", str(round(his_loss[bestid],4)))

    #target_model = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
    SAVE_PATH = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
    print("### loading model is:",SAVE_PATH ,'###')
    #engine.model.load_state_dict(torch.load(args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"))
    checkpoint = torch.load(SAVE_PATH)
    engine.model.load_state_dict(checkpoint['model_state_dict'])
    engine.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    print("### Loading Model finished ###")
    print("### The valid loss on loding model is", str(round(loss,4)))


    #----------------- 此內不能刪掉，不然會crash -----------------------#
    #valid data
    outputs = []
    realy = torch.Tensor(dataloader['y_val']).to(device)
    realy = realy.transpose(1,3)[:,0,:,:]
    print('#realy', realy.shape)

    for iter, (x, y, x_1, x_2) in enumerate(dataloader['val_loader'].get_iterator()):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1,3)

        testx_1 = torch.Tensor(x_1).to(device)
        testx_1 = testx_1.transpose(1, 3)
        testx_2 = torch.Tensor(x_2).to(device)
        testx_2 = testx_2.transpose(1, 3)

        with torch.no_grad():
            preds = engine.model(testx,testx_1,testx_2)
            preds = preds.transpose(1,3)  # 64,1,6,12

        outputs.append(preds.squeeze()) # 64,1,6,12 ->squeeze()->64,6,12

    yhat = torch.cat(outputs,dim=0)
    yhat = yhat[:realy.size(0),...]  # 5240,6,12
    print('# cat valid preds', yhat.shape)


    #pred = scaler.inverse_transform(yhat)

    '''
    pred = yhat      # 5240,6,12])
    for i in range(args.num_nodes):
      pred[:,i,:] = scaler_list[i].inverse_transform(pred[:,i,:])
    '''
    pred = scaler.inverse_transform(yhat)

    vmae, vmape, vrmse,vsmape = metric(pred,realy)
    print("valid mape",vmape)
    #----------------- 此內不能刪掉，不然會crash -----------------------#


    #test data
    outputs = []
    realy = torch.Tensor(dataloader['y_test']).to(device)
    realy = realy.transpose(1, 3)[:, 0, :, :]

    for iter, (x, y, x_1, x_2) in enumerate(dataloader['test_loader'].get_iterator()):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)

        testx_1 = torch.Tensor(x_1).to(device)
        testx_1 = testx_1.transpose(1, 3)
        testx_2 = torch.Tensor(x_2).to(device)
        testx_2 = testx_2.transpose(1, 3)

        with torch.no_grad():
            preds = engine.model(testx, testx_1, testx_2)
            preds = preds.transpose(1, 3)
        outputs.append(preds.squeeze())

    yhat = torch.cat(outputs, dim=0)
    yhat = yhat[:realy.size(0), ...]  #10478, 6, 12
    print('# cat test preds', yhat.shape)

    mae = []
    mape = []
    rmse = []
    smape = []
    for i in range(args.seq_out_len):
        #pred = scaler.inverse_transform(yhat[:, :, i])
        '''
        pred = yhat             # 10478, 6, 12
        for j in range(args.num_nodes):
          pred[:,j,i] = scaler_list[j].inverse_transform(pred[:,j,i])

        real = realy[:, :, i]
        real = realy[:, :, i]
        metrics = metric(pred[:,:,i], real)
        #print("#Predict:",i,", test maps", metrics[1])
        '''
        pred = scaler.inverse_transform(yhat[:, :, i])
        #pred = yhat             # 10478, 6, 12
        #for j in range(args.num_nodes):
        #  pred[:,j,i] = scaler_list[j].inverse_transform(pred[:,j,i])

        real = realy[:, :, i]
        #real = realy[:, :, i]
        #metrics = metric(pred[:,:,i], real)
        #print("#Predict:",i,", test maps", metrics[1])
        metrics = metric(pred, real)

        log = 'Evaluate best model on test data for horizon {:d}, Test MAE: {:.4f}, Test MAPE: {:.4f}, Test RMSE: {:.4f}'
        print(log.format(i + 1, metrics[0], metrics[1], metrics[2]))
        mae.append(metrics[0])
        mape.append(metrics[1])
        rmse.append(metrics[2])
        smape.append(metrics[3])

    log = '{:.2f}	{:.2f}	{:.4f}	{:.4f}	'
    print( "##### exp" + str(args.expid) + "_" + str(runid)+'	',
          log.format(mae[0], rmse[0], smape[0], mape[0]),
          log.format(mae[2], rmse[2], smape[2], mape[2]),
          log.format(mae[5], rmse[5], smape[5], mape[5]),
          log.format(mae[11], rmse[11], smape[11], mape[11]),
         )

    ### Drawing Loss Diagram ###
    fig = plt.figure(figsize=(10, 6), dpi=600)
    plt.plot(checkpoint['train_loss'], label="train loss")
    plt.plot(checkpoint['valid_loss'], label="valid loss")
    plt.legend(loc="upper right")
    plt.title('#Loss of Training', fontsize=20)
    plt.ylabel("MAPE", fontsize=14)
    plt.xlabel("Epochs", fontsize=14)
    plt.show()

    return vmae, vmape, vrmse,vsmape, mae, mape, rmse,smape

if __name__ == "__main__":

    vmae = []
    vmape = []
    vrmse = []
    vsmape = []
    mae = []
    mape = []
    rmse = []
    smape = []
    for i in range(args.runs):
        vm1, vm2, vm3,vm4, m1, m2, m3, m4 = main(i)
        vmae.append(vm1)
        vmape.append(vm2)
        vrmse.append(vm3)
        vsmape.append(vm4)
        mae.append(m1)
        mape.append(m2)
        rmse.append(m3)
        smape.append(m4)

    mae = np.array(mae)
    mape = np.array(mape)
    rmse = np.array(rmse)
    smape = np.array(smape)

    amae = np.mean(mae,0)
    amape = np.mean(mape,0)
    armse = np.mean(rmse,0)
    asmape = np.mean(smape,0)

    smae = np.std(mae,0)
    s_mape = np.std(mape,0)
    srmse = np.std(rmse,0)
    s_smape = np.std(smape,0)

    print('\n\nResults for 10 runs\n\n')
    #valid data
    print('valid\tMAE\tRMSE\tMAPE')
    log = 'mean:\t{:.4f}\t{:.4f}\t{:.4f}'
    print(log.format(np.mean(vmae),np.mean(vrmse),np.mean(vmape)))
    log = 'std:\t{:.4f}\t{:.4f}\t{:.4f}'
    print(log.format(np.std(vmae),np.std(vrmse),np.std(vmape)))
    print('\n\n')
    #test data
    print('test|horizon\tMAE-mean\tRMSE-mean\tMAPE-mean\tMAE-std\tRMSE-std\tMAPE-std')
    for i in [2,5,11]:
        log = '{:d}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}'
        print(log.format(i+1, amae[i], armse[i], amape[i], smae[i], srmse[i], s_mape[i]))


# 新增區段